Cluster the orange ball using Single Guassian

In [1]:
# import packages
import cv2
import os
import numpy as np
import math
import scipy
import matplotlib.pyplot as plt
import sys
# from datetime import datetime # for benchmarking purpose

# load data
train_dir = "train_images"# path to the train image dataset
test_dir = "test_images"# path to the train image dataset
# output directory
output_dir = "results"
# User defined threshold
tau = 0.00000017
prior = 0.5

In [2]:
def cal_mean_cov(img):
    l, w, h = img.shape
    mean = [[np.sum(img[:,:,0])/(l*w)],[np.sum(img[:,:,1])/(l*w)],[np.sum(img[:,:,2])/(l*w)]]
    cov = np.zeros((3,3),)
    #R_value = []
    for width in range(len(img[:,0,0])):
        for length in range(len(img[0,:,0])):
            RGB_value = [[img[width][length][0]],[img[width][length][1]],[img[width][length][2]]]
            cov = cov + (np.asmatrix(RGB_value) - np.asmatrix(mean))@(np.asmatrix(RGB_value) - np.asmatrix(mean)).T
            #R_value.append(img[width][length][0])
    cov = cov/(l*w)
    #print("size of image: ", l*w)
    #print("variance for R is: ", np.var(R_value))
    return mean,cov

In [3]:
def cal_mean_cov_vectorized(X):
    N, D = X.shape
    mean = X.mean(axis = 0) # compute mean
    cov = np.matmul((X - mean).T, (X - mean))/(N-1) # compute covariance
    return mean, cov

In [4]:
# return Nx3 array
def extract_orange_pixels():
    # store orange pixels, each row is a pixel
    orange_pixels = np.array([])

    for img_name in os.listdir(train_dir):
        if "mask" in img_name:
            continue
        img = cv2.imread(os.path.join(train_dir, img_name))
        # load mask for it
        img_mask = cv2.imread(os.path.join(train_dir, img_name.split(".")[0]+"_mask.png"))
        # reshape to num of rows = num of pixels, num of column = 3 (BGR)
        X = img.transpose(2,0,1).reshape(3,-1).T 
        # reshape and sum mask to 1d array
        X_mask = img_mask.transpose(2,0,1).reshape(3,-1).T.sum(1) 
        # if empty array
        if orange_pixels.size == 0:
            orange_pixels = X[X_mask>50] # get pixels that are not black in mask
        else:
            orange_pixels = np.append(orange_pixels, X[X_mask>20], axis =0)

    return orange_pixels

# print orange pixels
# np.set_printoptions(threshold=np.inf)
# print(extract_orange_pixels())
# cv2.imshow("image_name", extract_orange_pixels().reshape(32, -1, 3))
# cv2.waitKey(0)

In [5]:
# train on orange pixels
# param X is orange pixels, Nx3 array 
def train_on_orange_pixels(X):
    mean, cov = cal_mean_cov_vectorized(X)   
    return mean, cov

In [6]:
# test on test images
# param: mean and cov of all orange pixels
def test(orange_mean, orange_cov):
    for img_name in os.listdir(test_dir):
        img = cv2.imread(os.path.join(test_dir, img_name))
        l, w, h = img.shape # original shape of 2D image
        X = img.transpose(2,0,1).reshape(3,-1).T # reshape to num of rows = num of pixels, num of column = 3 (RGB)
        N, D = X.shape
        #img = X.reshape(l, w, -1) # reshape back to 2d image
    
        # calculate likelihood using gaussian distribution
        # each pixel is row of X
        constant_in_likelihood = 1/(math.sqrt(((2*math.pi)**3)* np.linalg.det(orange_cov))) 
        sigma_inv = np.linalg.inv(orange_cov)
        X2 = X-orange_mean
        exponent = (-0.5)*(np.dot(X2, sigma_inv) * X2).sum(1) 
        likelihood = constant_in_likelihood * np.exp(exponent)

        # posterior
        prior = 0.5
        posterior = prior* likelihood 
        
        # mask (reshape back to 2D image)
        mask = posterior.reshape(l, w) 
        #print(mask)
        
        # apply mask
        img[mask < tau] = 0

        ##  show masked img
        image_name = os.path.join(output_dir,"single_gaussian_"+ str(img_name))
        cv2.imshow(image_name, img)
        if not os.path.exists(output_dir):
            os.mkdir(output_dir)
        cv2.imwrite(image_name, img)
        cv2.waitKey(0)
        print("Finish Generating mask for image ", str(img_name))
    print("Complete Process All Images")


In [7]:
orange_mean, orange_cov = train_on_orange_pixels(extract_orange_pixels())
print("orange_mean: "+ str(orange_mean)) # BGR, not RGB
print("orange_cov: \n" + str(orange_cov))
test(orange_mean, orange_cov)

orange_mean: [ 84.96744737 111.55088526 203.92534504]
orange_cov: 
[[1324.48893677 1413.38440317  773.44964351]
 [1413.38440317 1737.22644502 1260.45744149]
 [ 773.44964351 1260.45744149 1544.4586298 ]]
Finish Generating mask for image  99.jpg
Finish Generating mask for image  248.jpg
Finish Generating mask for image  208.jpg
Finish Generating mask for image  144.jpg
Finish Generating mask for image  256.jpg
Finish Generating mask for image  223.jpg
Finish Generating mask for image  68.jpg
Finish Generating mask for image  91.jpg
Finish Generating mask for image  231.jpg
Finish Generating mask for image  76.jpg
Finish Generating mask for image  168.jpg
Finish Generating mask for image  152.jpg
Finish Generating mask for image  216.jpg
Finish Generating mask for image  106.jpg
Finish Generating mask for image  264.jpg
Finish Generating mask for image  192.jpg
Finish Generating mask for image  114.jpg
Finish Generating mask for image  200.jpg
Finish Generating mask for image  121.jpg
Fin

In [8]:
v = "single_gaussian_result\masked_train_images\106.jpg"
v.split('\\')

['single_gaussian_result', 'masked_train_imagesF.jpg']

In [9]:
matrixSize = 3
A = scipy.random.rand(matrixSize,matrixSize)
B = np.dot(A,A.transpose())
print ('random positive semi-define matrix for today is', B)


random positive semi-define matrix for today is [[0.42353427 0.60524585 0.94764823]
 [0.60524585 1.56792883 1.76721325]
 [0.94764823 1.76721325 2.6248064 ]]
